# Reactive transport

In this tutorial we investigate the reactive transport problem.

## Exercise 3: transport and precipitation of chemical species

Let $\Upsilon=(0,1)^2$ with boundary $\partial \Upsilon$ and outward unit normal ${\nu}$.
We define also $(0, T)$ the time interval, being $T$ the final time.

Given 
$k = I$ the matrix permeability, we want to solve the following problem: find $p$ such that
$$
\nabla \cdot (- k\nabla p) = 0
\quad \text{in } \Upsilon
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{right} \Upsilon \qquad p = 1 \text{ on } \partial_{left} \Upsilon \qquad \nu \cdot k \nabla p = 0 \text{ on } \partial_{top} \Upsilon \cup \partial_{bottom} \Upsilon$$

Given the flux $q = - k \nabla p$ computed by the previous model, we want to solve the following problem: find $c$ and $w$, called mobile and immobile species respectively, such that
$$
\begin{cases}
\partial_t c + \nabla \cdot (qc) - r(c, w)= 0\\
\partial_t w + r(c, w) = 0
\end{cases}
\quad \text{in } \Upsilon \times (0, T)
$$
where the reaction function $r$ is defined as
\begin{gather*}
r(c, w) = 
\begin{cases}
k_d (1 - \Omega) & \text{if } \Omega < 1 \text{ and } w > 0\\
k_p (1-\Omega) & \text{if } \Omega > 1
\end{cases}
\end{gather*}
here $\Omega$ is the product of solubility given by $\Omega = c / c_{eq}$, and the latter being set to $0.5$. And $k_d$ and $k_p$ are the reaction coefficients that are associated to the dissolution and precipitation chemical processes.



The boundary conditions for the previous model are set on the inflow of the domain:
$$ c = c_I \quad \text{ on } \partial \Upsilon \times (0, T)$$
and initial condition for the concentration
$$ c(x, 0) = c_0(x) \quad \text{ in } \Upsilon $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

Before creating the grid we import NumPy, the SciPy sparse library and PorePy.

In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp

We specify number of cells in each dimension and the physical size of the domain. Then we create a Cartesian grid and compute geometric properties such as face centers, cell volumes etc.

In [2]:
dim = 2
N = [20] * dim
phys_dims = [1] * dim

sd = pp.CartGrid(N, phys_dims)
sd.compute_geometry()

We first need to solve the Darcy problem, so declare its data.

In [3]:
# Permeability
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))

# define outflow and inflow type boundary conditions, left and right boundary
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
b_face_centers = sd.face_centers[:, b_faces]

outflow = np.isclose(b_face_centers[0, :], 1)
inflow = np.isclose(b_face_centers[0, :], 0)

# define the labels and values for the boundary faces
labels = np.array(["neu"] * b_faces.size)
bc_val = np.zeros(sd.num_faces)

labels[np.logical_or(inflow, outflow)] = "dir"
bc_val[b_faces[inflow]] = 1

bc = pp.BoundaryCondition(sd, b_faces, labels)

We now set the data for the Darcy problem

In [4]:
flow_key = "flow"

# Collect all parameters in a dictionary
flow_data = {
    pp.PARAMETERS: {
        flow_key: {
            "second_order_tensor": perm,
            "bc_values": bc_val,
            "bc": bc,
        }
    },
    pp.DISCRETIZATION_MATRICES: {flow_key: {}},
}

We now solve the Darcy problem by using the MPFA scheme.

In [5]:
# construct the lhr and rhs from the discretization of the diffusion operator
mpfa = pp.Mpfa(flow_key)
mpfa.discretize(sd, flow_data)
A, b = mpfa.assemble_matrix_rhs(sd, flow_data)

# solve the problem
cell_p = sps.linalg.spsolve(A, b)

# now data contains the discretization matrices build from MPFA
mat_discr = flow_data[pp.DISCRETIZATION_MATRICES][flow_key]

q = mat_discr["flux"] @ cell_p + mat_discr["bound_flux"] @ bc_val

And we export the corresponding solutions.

In [6]:
save = pp.Exporter(sd, "sol_p", folder_name="ex3")
exp_p = save.write_vtu([("cell_p", cell_p)])

We now consider the transport problem where now the advective field is the one computed from the Darcy problem. First we set the data.

In [7]:
# Transport problem
transport_key = "transport"
delta_t = 0.01
num_steps = 100

# Set in the data file the flux
bc_val = np.zeros(sd.num_faces)
bc_val[b_faces[inflow]] = 1

# Collect all parameters in a dictionary
transport_data = {
    pp.PARAMETERS: {
        transport_key: {
            "darcy_flux": q,
            "bc_values": bc_val,
            "bc": bc,
        }
    },
    pp.DISCRETIZATION_MATRICES: {transport_key: {}},
}

We now construct the upwind matrix and the mass matrix.

In [8]:
# create the upwind and mass matrices
upwind = pp.Upwind(transport_key)

# discretize and get the matrices
upwind.discretize(sd, transport_data)

U, b_upwind = upwind.assemble_matrix_rhs(sd, transport_data)
M = sps.diags(sd.cell_volumes)

Finally, by using the implicit Euler we compute the concentration that is transported in the porous medium. Since the reaction term is nonlinear we employ a splitting strategy. My calling $M$ and $U$ the associated mass and upwind matrices and $R$ the reaction term we consider the following strategy for each time step $n$. Instead of
solving the following nonlinear problem
\begin{gather*}
    \begin{cases}
    &(M + \Delta t U) c^{n+1} - \Delta t M R(c^{n+1}, w^{n+1}) = M c^n\\
    &M w^{n+1}  + \Delta t M R(c^{n+1}, w^{n+1}) = M w^n
    \end{cases}
\end{gather*}
we solve sequentially the following semi-implicit scheme
\begin{align*}
    &\text{first step}
    &&(M + \Delta t U) c^{*} = M c^n
    \\
    &\text{second step} &&
    \begin{cases}
    &c^{n+1} = c^{*} + \Delta t R(c^{*}, w^{n})\\
    &w^{n+1} = w^n - \Delta t R(c^{*}, w^{n})
    \end{cases}
\end{align*}

In [9]:
# Initial condition and exporter
c = np.zeros(sd.num_cells)
w = (
    0.5
    * (sd.cell_centers[0] > 0.4)
    * (sd.cell_centers[0] < 0.6)
    * (sd.cell_centers[1] > 0.4)
    * (sd.cell_centers[1] < 0.6)
)
save = pp.Exporter(sd, "sol_c", folder_name="ex3")
save.write_vtu([("conc", c), ("prec", w)], time_step=0)

kd = 1
kp = 1
c_eq = 0.5

# IE
S = M + delta_t * U
for i in np.arange(num_steps):
    c_star = sps.linalg.spsolve(S, M @ c - delta_t * b_upwind)

    # with a splitting perform the chemical part cell by cell
    omega = c_star / c_eq
    rd = kd * (1 - omega) * (w > 0) * (omega < 1)
    rp = kp * (1 - omega) * (omega > 1)
    c = c_star + delta_t * (rd + rp)
    w = w + delta_t * (-rd - rp)

    save.write_vtu([("conc", c), ("prec", w)], time_step=(i + 1))

# export the main pvd file
time = np.arange((num_steps + 1)) * delta_t
save.write_pvd(time)

In [10]:
# Consistency check
assert np.isclose(np.linalg.norm(c), 14.011425800214226)
assert np.isclose(np.linalg.norm(w), 6.882322992544662)